In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

repo_root = Path.cwd().parent  # docs/ -> repo root
sys.path.insert(0, str(repo_root))
# import tests.fixtures.test_case_catalog as cases
import tests.fixtures.test_case_catalog as cases
import tests.utils_for_testing as tst_ut


# compilation

In [3]:
case = cases.Local_TestCases.retrieve_norfolk_all_models_test_case(start_from_scratch=True)
analysis = case.analysis
system = analysis._system

system.compile_TRITON_SWMM(recompile_if_already_done_successfully=True, redownload_triton_swmm_if_exists=True, verbose = True)

[Download] Cloning TRITON-SWMM to /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/triton


Cloning into 'triton'...
Note: switching to '15eb18a5d25afe5da295cb4b559a62669dbe5bc3'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 15eb18a output directory fixes   1. SWMM Output Directory Fix
Submodule 'external/kokkos' (https://github.com/kokkos/kokkos.git) registered for path 'external/kokkos'
Submodule 'external/yaml-cpp' (https://github.com/jbeder/yaml-cpp.git) registered for path 'external/yaml-cpp'
Cloning into '/home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/triton/external/

Submodule path 'external/kokkos': checked out 'dc1bf738ea1a5676225c5592409af9d77a100a43'
Submodule path 'external/yaml-cpp': checked out 'a83cd31548b19d50f3f983b069dceb4f4d50756d'

Compiling CPU Backend
[CPU] Starting compilation...
[CPU]   Script: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/compile_cpu.sh
[CPU] ✓ Compilation successful!


In [5]:
system.compile_TRITON_only(recompile_if_already_done_successfully=True, verbose = True)


Compiling TRITON-only CPU Backend
[TRITON-only CPU] Starting compilation...
[TRITON-only CPU]   Script: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/triton/compile_triton_only_cpu.sh


[TRITON-only CPU] ✓ Compilation successful!


# single sim test case

In [2]:
case = cases.Local_TestCases.retrieve_norfolk_all_models_test_case(start_from_scratch=True)
analysis = case.analysis
system = analysis._system

In [8]:
event_iloc = 0
proc = analysis._retrieve_sim_run_processing_object(event_iloc)
paths = proc.scen_paths

swmm_node_ts = paths.output_swmm_only_node_time_series
swmm_link_ts = paths.output_swmm_only_link_time_series
tritonswmm_node_ts = paths.output_tritonswmm_node_time_series
tritonswmm_link_ts = paths.output_tritonswmm_link_time_series

In [9]:
import xarray as xr
ds_swmm_nodes = xr.open_dataset(swmm_node_ts)
ds_swmm_links = xr.open_dataset(swmm_link_ts)
ds_tritonswmm_nodes = xr.open_dataset(tritonswmm_node_ts)
ds_tritonswmm_links = xr.open_dataset(tritonswmm_link_ts)

<xarray.DataArray 'max_flow_cms' (node_id: 1133)> Size: 9kB
[1133 values with dtype=float64]
Coordinates:
  * node_id  (node_id) <U10 45kB '05-0088' '07-0077' '11-0057' ... 'UN90' 'UN91'

<xarray.DataArray 'max_flow_cms' (node_id: 1135)> Size: 9kB
[1135 values with dtype=float64]
Coordinates:
  * node_id  (node_id) <U10 45kB '05-0088' '07-0077' ... 'UN91' 'dummy_of'

In [ ]:
analysis = norfolk_all_models_analysis_cached

if "swmm" not in tst_ut.get_enabled_model_types(analysis):
    pytest.skip("SWMM-only model not enabled")
if "tritonswmm" not in tst_ut.get_enabled_model_types(analysis):
    pytest.skip("TRITON-SWMM model not enabled")

for event_iloc in analysis.df_sims.index:
    proc = analysis._retrieve_sim_run_processing_object(event_iloc)
    paths = proc.scen_paths

    swmm_node_ts = paths.output_swmm_only_node_time_series
    swmm_link_ts = paths.output_swmm_only_link_time_series
    tritonswmm_node_ts = paths.output_tritonswmm_node_time_series
    tritonswmm_link_ts = paths.output_tritonswmm_link_time_series

    # Verify all required output files exist
    tst_ut.assert_file_exists(swmm_node_ts, "SWMM-only node timeseries")
    tst_ut.assert_file_exists(swmm_link_ts, "SWMM-only link timeseries")
    tst_ut.assert_file_exists(tritonswmm_node_ts, "TRITON-SWMM node timeseries")
    tst_ut.assert_file_exists(tritonswmm_link_ts, "TRITON-SWMM link timeseries")

    ds_swmm_nodes = xr.open_dataset(swmm_node_ts)
    ds_swmm_links = xr.open_dataset(swmm_link_ts)
    ds_tritonswmm_nodes = xr.open_dataset(tritonswmm_node_ts)
    ds_tritonswmm_links = xr.open_dataset(tritonswmm_link_ts)

    # Node/link ids should be present in the TRITON-SWMM datasets
    swmm_node_ids = set(ds_swmm_nodes["node_id"].values.tolist())
    swmm_link_ids = set(ds_swmm_links["link_id"].values.tolist())
    tritonswmm_node_ids = set(ds_tritonswmm_nodes["node_id"].values.tolist())
    tritonswmm_link_ids = set(ds_tritonswmm_links["link_id"].values.tolist())

    missing_nodes = swmm_node_ids - tritonswmm_node_ids
    missing_links = swmm_link_ids - tritonswmm_link_ids

    if missing_nodes:
        pytest.fail(
            f"TRITON-SWMM node_ids missing {len(missing_nodes)} SWMM-only nodes."
        )
    if missing_links:
        pytest.fail(
            f"TRITON-SWMM link_ids missing {len(missing_links)} SWMM-only links."
        )

    # Timestep counts should match
    if len(ds_swmm_nodes["date_time"]) != len(ds_tritonswmm_nodes["date_time"]):
        pytest.fail("Node time series timestep counts do not match")
    if len(ds_swmm_links["date_time"]) != len(ds_tritonswmm_links["date_time"]):
        pytest.fail("Link time series timestep counts do not match")

    # Data variables should match (order-agnostic)
    if set(ds_swmm_nodes.data_vars) != set(ds_tritonswmm_nodes.data_vars):
        pytest.fail("Node time series data variables do not match")
    swmm_link_vars = set(ds_swmm_links.data_vars)
    tritonswmm_link_vars = set(ds_tritonswmm_links.data_vars)

    # Normalize known naming differences before comparing
    swmm_link_vars = tst_ut.normalize_swmm_link_vars(swmm_link_vars)
    tritonswmm_link_vars = tst_ut.normalize_swmm_link_vars(tritonswmm_link_vars)

    if swmm_link_vars != tritonswmm_link_vars:
        pytest.fail("Link time series data variables do not match")

## compilation

## scenario prep

In [ ]:
analysis.run_prepare_scenarios_serially(
    overwrite_scenario_if_already_set_up=True, rerun_swmm_hydro_if_outputs_exist=True
)
tst_ut.assert_scenarios_setup(analysis)

2026-02-05 17:13:49,378 [INFO] Loading system configuration from /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/tests.yaml
2026-02-05 17:13:49,381 [INFO] Loading analysis configuration from /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/all_models/all_models.yaml
2026-02-05 17:13:49,388 [INFO] Preparing scenario 0
[Scenario 0] Using CPU backend

... SWMM Version 5.2.4
 o  Retrieving project data
 o  Simulating day: 0     hour: Simulation complete           
... Run CompleteScenario preparation complete
2026-02-05 17:13:52,625 [INFO] Scenario 0 prepared successfully


## run

In [5]:
analysis.run_sims_in_sequence(pickup_where_leftoff=False)
tst_ut.assert_scenarios_run(analysis)
for model_type in tst_ut.get_enabled_model_types(analysis):
    tst_ut.assert_model_simulation_run(analysis, model_type)

/home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/all_models/sims/0-event_id.0


Failed: One or more simulations failed to run

In [11]:
case.analysis.process_all_sim_timeseries_serially()

Processing run results for scenario 0
Processed TRITON outputs for scenario 0
##################################
Found problem. Orifice conduits do not return max velocity or max over full flow. Filling with empty string
Normal row vs. problem row:
  0                    CONDUIT     0.000     0  00:01      0.00    0.00    0.00

  756                  ORIFICE     0.000     0  00:00                      0.00

Properly parsed values:
['756', 'ORIFICE', '0.000', '0', '00:00', '', '', '0.00\n']
Converted variable to datatype = type, <class 'str'>
Converted variable to datatype = type, <class 'str'>
Processed SWMM outputs for scenario 0


# multi-sim case

In [ ]:
case = cases.Local_TestCases.retrieve_norfolk_multi_sim_test_case(start_from_scratch=False)
analysis = case.analysis

In [ ]:
# analysis = norfolk_multi_sim_analysis

result = analysis.submit_workflow(
    mode="local",
    process_system_level_inputs=True,
    overwrite_system_inputs=True,
    compile_TRITON_SWMM=True,
    recompile_if_already_done_successfully=True,
    prepare_scenarios=True,
    overwrite_scenario_if_already_set_up=True,
    rerun_swmm_hydro_if_outputs_exist=True,
    process_timeseries=True,
    which="both",
    clear_raw_outputs=True,
    overwrite_outputs_if_already_created=True,
    compression_level=5,
    pickup_where_leftoff=False,
    dry_run=False,
    verbose=True,
)

[Snakemake] Submitting workflow in local mode
[Snakemake] Snakefile generated: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/Snakefile
[Snakemake] Running workflow locally with Snakemake
[Snakemake] DRY RUN
[Snakemake] Using dynamic config from: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/.snakemake_profile/local


[Snakemake] Workflow completed successfully
[Snakemake] Running workflow locally with Snakemake
[Snakemake] Using dynamic config from: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/.snakemake_profile/local
[Snakemake] Workflow completed successfully


In [9]:
analysis.SWMM_node_summary

ValueError: could not open file because it does not exist: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/multi_sim/SWMM_nodes.zarr. Run analysis.consolidate_[SWMM/TRITON]_outputs().

# sensitivity analysis

In [ ]:
nrflk_cpu_sensitivity = cases.Local_TestCases.retrieve_norfolk_cpu_config_sensitivity_case(
        start_from_scratch=True
    )
analysis = nrflk_cpu_sensitivity.analysis

In [ ]:
result = analysis.submit_workflow(
    mode="local",
    process_system_level_inputs=True,
    overwrite_system_inputs=True,
    compile_TRITON_SWMM=True,
    recompile_if_already_done_successfully=True,
    prepare_scenarios=True,
    overwrite_scenario_if_already_set_up=True,
    rerun_swmm_hydro_if_outputs_exist=True,
    process_timeseries=True,
    which="both",
    clear_raw_outputs=True,
    overwrite_outputs_if_already_created=True,
    compression_level=5,
    pickup_where_leftoff=False,
    dry_run=True,
    verbose=True,
)

[Snakemake] Submitting sensitivity analysis workflow in local mode
[Snakemake] Generated master Snakefile: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/cpu_config_sensitivity/Snakefile
[Snakemake] Created required directories (_status, logs/sims)
[Snakemake] Running workflow locally with Snakemake
[Snakemake] Using dynamic config from: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/tests/cpu_config_sensitivity/.snakemake_profile/local
[Snakemake] Workflow completed successfully


In [ ]:
case = cases.Local_TestCases.retrieve_norfolk_cpu_config_sensitivity_case(start_from_scratch=False)
analysis = case.analysis

In [ ]:
analysis.sensitivity.compile_TRITON_SWMM_for_sensitivity_analysis()
assert analysis.sensitivity.compilation_successful == True

In [ ]:
analysis.sensitivity.prepare_scenarios_in_each_subanalysis(concurrent=True)
assert analysis.log.all_scenarios_created.get() == True

In [ ]:
analysis.sensitivity.run_all_sims(pickup_where_leftoff=False, concurrent=False)
# assert analysis.log.all_sims_run.get() == True